## KR260 HLS QPSK Demod


### Plot function for use in this notebook
The first code block below defines a function that we will use for plotting data throughout this notebook. Note that the function has a `n_samples` argument so that we can limit the number of samples to plot. Plotting more than a few thousand samples can be very slow and consume a lot of RAM.

### Requirements
Install the following:

```bash
pip install ipympl
pip install ipywidgets
pip install jupyter_bokeh
```

# Initilize

#### Bokeh Plot Function

In [23]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, show
import numpy as np
import math


def plot_time(in_signal,out_signal=None):
    t = np.linspace(0,len(in_signal),len(in_signal))
    output_notebook()
    p = figure(title = "Input & Output Signal")
    
    if out_signal is not None:
        p.line(t,out_signal,legend_label="Output Signal",line_color="red",line_width=1)
    p.line(t,in_signal,legend_label="Input Signal",line_color="blue",line_width=3)
    show(p)

#### Load Overlay

In [24]:
from pynq import Overlay
from pynq import allocate
from pynq import MMIO
import pynq.lib.dma
import time
import kria
import timestamp_regmap
import led_regmap
import qpsk_regmap


# Load the overlay
overlay = Overlay('/home/root/jupyter_notebooks/kr260_hls_qpsk_demod/kr260_hls_qpsk_demod.bit')

# Assign blocks to short vars
dma          = overlay.axi_dma_0
led_module   = overlay.led_reg_0
timestamp    = overlay.Timestamp_0
qpsk_demod   = overlay.QPSK_Demod_Top_0

##### Print out the register map for DMA and HLS Gain Blocks

In [ ]:
overlay.ip_dict

{'axi_dma_0': {'type': 'xilinx.com:ip:axi_dma:7.1',
  'mem_id': 'S_AXI_LITE',
  'memtype': 'REGISTER',
  'gpio': {},
  'interrupts': {},
  'parameters': {'C_S_AXI_LITE_ADDR_WIDTH': '10',
   'C_S_AXI_LITE_DATA_WIDTH': '32',
   'C_DLYTMR_RESOLUTION': '125',
   'C_PRMRY_IS_ACLK_ASYNC': '0',
   'C_ENABLE_MULTI_CHANNEL': '0',
   'C_NUM_MM2S_CHANNELS': '1',
   'C_NUM_S2MM_CHANNELS': '1',
   'C_INCLUDE_SG': '0',
   'C_SG_INCLUDE_STSCNTRL_STRM': '0',
   'C_SG_USE_STSAPP_LENGTH': '0',
   'C_SG_LENGTH_WIDTH': '23',
   'C_M_AXI_SG_ADDR_WIDTH': '32',
   'C_M_AXI_SG_DATA_WIDTH': '32',
   'C_M_AXIS_MM2S_CNTRL_TDATA_WIDTH': '32',
   'C_S_AXIS_S2MM_STS_TDATA_WIDTH': '32',
   'C_MICRO_DMA': '0',
   'C_INCLUDE_MM2S': '1',
   'C_INCLUDE_MM2S_SF': '1',
   'C_MM2S_BURST_SIZE': '16',
   'C_M_AXI_MM2S_ADDR_WIDTH': '32',
   'C_M_AXI_MM2S_DATA_WIDTH': '32',
   'C_M_AXIS_MM2S_TDATA_WIDTH': '32',
   'C_INCLUDE_MM2S_DRE': '0',
   'C_INCLUDE_S2MM': '0',
   'C_INCLUDE_S2MM_SF': '1',
   'C_S2MM_BURST_SIZE': '16',
  

##### Display Time Stamp Register
The time stamp is burned into the FPGA during the build process

In [25]:
print("FPGA Build Timestamp:  " + kria.get_timestamp_str(timestamp))

FPGA Build Timestamp:  2025/3/9 10:49:32


##### Toggle USER_LED[1:0] on/off

In [11]:

    led_module.mmio.write(led_regmap.RegMap.USER_LEDS_ADDR,0x1)
    time.sleep(0.25)
    led_module.mmio.write(led_regmap.RegMap.USER_LEDS_ADDR,0x2)
    time.sleep(0.25)
led_module.mmio.write(led_regmap.RegMap.USER_LEDS_ADDR,0x0)

### Configure the QPSK Demod Control Register

Want to enable auto restart, so the FPGA block will always be read to receive samples


In [26]:
fin_tBits, fin_fBits = kria.get_format(qpsk_demod,qpsk_regmap.RegMap.F_IN_ADDR)
print("F_IN : (" + str(fin_tBits) + ", " + str(fin_fBits) + ")")
fout_tBits, fout_fBits = kria.get_format(qpsk_demod,qpsk_regmap.RegMap.F_OUT_ADDR)
print("F_IN : (" + str(fout_tBits) + ", " + str(fout_fBits) + ")")

F_IN : (16, 12)
F_IN : (16, 12)


### Open Modulated Data Samples and Convert to INT16 I/Q Samples

In [27]:
mod_samps = []
with open('0xDEADBEEF_Rx_Samps.dat') as mod:
    mod_samps = mod.readlines()
mod_samps = [line.rstrip('\n') for line in mod_samps]

mod_samps = np.array(mod_samps)
mod_samps = mod_samps.astype('float64')*(2**fin_fBits)
mod_samps = mod_samps.astype(np.int32)
xi = mod_samps[0::2]
xq = mod_samps[1::2]
xiq = (xi<<16) + xq # pack I/Q 32-bit

print("Number of I Samples Read: " + str(len(xi)))
print("Number of Q Samples Read: " + str(len(xq)))
print("Number Symbols          : " + str(len(xq)/16))
plot_time(xi)
#print(hex(xi[1:10]))

Number of I Samples Read: 2432
Number of Q Samples Read: 2432
Number Symbols          : 152.0


Loading BokehJS ...

In [9]:
qpsk_demod.mmio.write(qpsk_regmap.RegMap.AP_CONTROL_ADDR,0)
print(qpsk_demod.mmio.read(qpsk_regmap.RegMap.AP_CONTROL_ADDR))

4


### DMA Transfer the Sample buffer from ARM Processor to the HLS QPSK Demodulator

In [28]:
# 16 Samples Per Symbol, therefore create read buffer /16
SamplesPerSym = 16
NumSamples = len(xiq)
NumSymbols = int(NumSamples/SamplesPerSym/4)
print("NumSamples = " + str(NumSamples))
print("NumSymboles = " + str(NumSymbols))

# Allocate buffers for the input and output signals
in_buffer = allocate(shape=(NumSamples,), dtype=np.int32)

# Copy the samples to the in_buffer
np.copyto(in_buffer,xiq)

# Clear the buffer write address
qpsk_demod.mmio.write(qpsk_regmap.RegMap.WR_RAM_ADDR_CTRL_ADDR,1)

# Start HLS QPSK Demodulator
qpsk_demod.mmio.write(qpsk_regmap.RegMap.AP_CONTROL_ADDR,0)
qpsk_demod.mmio.write(qpsk_regmap.RegMap.AP_CONTROL_ADDR,1)

# Trigger the DMA transfer and wait for the result
import time
start_time = time.time()
dma.sendchannel.transfer(in_buffer)
print("Done tranfer in_buffer")
dma.sendchannel.wait()
print("Done send channel wait")

stop_time = time.time()
hw_exec_time = stop_time-start_time

print('DMA Transfer Execution Time  :',hw_exec_time,' sec')

# Plot to the notebook
#plot_time(t,samples,2000,out_signal=out_buffer)

# Free the buffers
in_buffer.close()

print("Symbols in the Buffer: " + str(qpsk_demod.mmio.read(qpsk_regmap.RegMap.WR_RAM_ADDR)))


NumSamples = 2432
NumSymboles = 38
Done tranfer in_buffer
Done send channel wait
DMA Transfer Execution Time  : 0.003112316131591797  sec
Symbols in the Buffer: 152


In [9]:
print("WR Capture Address: " + str(qpsk_demod.mmio.read(qpsk_regmap.RegMap.WR_RAM_ADDR)))
print("RD Capture Address: " + str(qpsk_demod.mmio.read(qpsk_regmap.RegMap.RD_RAM_ADDR_ADDR)))
qpsk_demod.mmio.write(qpsk_regmap.RegMap.WR_RAM_ADDR_CTRL_ADDR,1)
print("WR Capture Address: " + str(qpsk_demod.mmio.read(qpsk_regmap.RegMap.WR_RAM_ADDR)))

WR Capture Address: 0
RD Capture Address: 151
WR Capture Address: 0


In [33]:
rd_len = qpsk_demod.mmio.read(qpsk_regmap.RegMap.WR_RAM_ADDR)
print("Number of Symbols to Read: " + str(rd_len))
dout = []
for i in range(rd_len):
    qpsk_demod.mmio.write(qpsk_regmap.RegMap.RD_RAM_ADDR_ADDR,i)
    dout.append(qpsk_demod.mmio.read(qpsk_regmap.RegMap.RD_RAM_DATA_ADDR))
    
for i in dout:
    print(str(hex(i)))

Number of Symbols to Read: 152
0xef30c100
0xefdeadbe
0xefdeadbe
0xefdeadbe
0xefdeadbe
0xefdeadbe
0xefdeadbe
0xefdeadbe
0x33deadbe
0x82
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
0x0
